In [1]:
import os
import time
import dotenv
dotenv.load_dotenv("../.env")
from swan import SwanAPI, SwanContract

# Initialize the Swan Service
# dev: XraxVKnsWn
# M2bPEWEkiE
# wallet: 0x5339595102d922AA0F8EfD6DCA9a2b406F57937a
# environment="https://swanhub-cali.swanchain.io"
swan_api = SwanAPI(api_key="M2bPEWEkiE")

from swan import MCSAPI
 
api_key = os.getenv("MCS_API")
mcs_api = MCSAPI(api_key)


In [2]:
hardwares = swan_api.get_hardware_config()

obj = [hardware for hardware in hardwares if hardware.name == 'G1ae.medium'][0]
print(obj)

In [3]:
from swan.object.source_uri import LagrangeSpace

lag = LagrangeSpace('0x61c3e03dbed55f5DE213732e816F8A8Fd6E9bfF0', 'NSFW_Detection', os.getenv("WALLET"), obj)
 
lag.get_space_info()

# Upload Directory to MCS
response = lag.generate_source_uri('swan_test', 'mar8s1', 'nsfw.json', mcs_client = mcs_api)

mar8s1: 3.35kB [00:00, 31.5kB/s]                   

In [6]:
print(lag.source_uri)

https://plutotest.acl.swanipfs.com/ipfs/QmVBtL7ffwg9wKfCyMjpjXwgSTQDvp1p8eVT2CtUznzufw


In [8]:
print([(hardware.name, hardware.id, hardware.region) for hardware in hardwares if hardware.name == 'G1ae.medium'][0])

('G1ae.medium', 13, ['Quebec-CA', 'Kowloon City-HK', 'Jiangsu-CN'])


In [13]:
import sys
sys.path.insert(0, '..')
 
import os
from dotenv import load_dotenv
load_dotenv()
 
from swan import SwanContract
 
pk = os.getenv('PK')
rpc = os.getenv('RPC')
 
c = SwanContract(pk, rpc)
 
# Test esimate lock revenue
r = c.estimate_payment(13, 2)
print(r)


22000000000000000000


In [14]:
 
# # Test get hardware info
# r = c.hardware_info(1)
# print(r)
 
# # Test get swan token balance
# r = c._get_swan_balance()
# print(r * 1e-18)
 
# # Test get gas
# r = c._get_swan_gas()
# print(r* 1e-18)
 
r = c._approve_swan_token(22000000000000000000)
print(r)
 
# gas_estimate = c.payment_contract.functions.lockRevenue('1', 1, 1).estimate_gas()
# print(gas_estimate)
 
# r = c.w3.eth.get_block('latest')['baseFeePerGas']
# print(r)
 
r = c.lock_revenue('1', 13, 1)
print(r)

0x7a6d3d4b3ea9b16f900da07d8e206057e822d7d50393bac87e553e6cc17128aa
0x584983609061d4dbb99e2ab8744e498284f812195db6196058f4bf5c36a6537a


In [ ]:
# Retrieve List of Hardwares

# for hardware in hardwares:
#     if hardware.name == 'C1ae.medium':
#         print(hardware.region, hardware.price)
        
# Deploy task
result = swan_api.deploy_task(
                              cfg_name='G1ae.medium', 
                              region='Quebec-CA', 
                              start_in=60*40, duration=3600*100, 
                              job_source_uri=lag.source_uri, 
                              paid=1100,
                              tx_hash="0xc3108306ae9890041fb376480a5d6c7e997903cd388c1061d5f0418ed5465a31",
                              wallet_address=os.getenv('WALLET')
                            )
print(result)
task_uuid = result['data']['task']['uuid']
print("Task UUID:", task_uuid)

In [21]:
import json

# Check task info
while True:
    info = swan_api.get_deployment_info(task_uuid=task_uuid)
    print(json.dumps(info, indent=4))
    if len(info['data']['jobs']) > 0:
        
        status = info['data']['jobs'][0]['status']
        print(status)
        job_res_url = info['data']['jobs'][0]['job_result_uri']
        job_real_url = info['data']['jobs'][0]['job_real_uri']
        print("Job Result URL: ", job_res_url)
        print("Job Real URL: ", job_real_url)
        
        # break
        if status != 'buildImage':
            break
        
    time.sleep(60)

{
    "data": {
        "computing_providers": [
            {
                "allowed_nodes": null,
                "autobid": 1,
                "city": "Montreal",
                "country": null,
                "created_at": "1706080831",
                "deleted_at": null,
                "last_active_at": null,
                "lat": 45.5075,
                "lon": -73.5887,
                "multi_address": "/ip4/38.140.46.60/tcp/8099",
                "name": "filezoo.com.cn",
                "node_id": "0493113e3cd4d0a02a3b9494059caef6f70ba1987c8867ea98b676d59f795f7dd2b304a3f46688b8b0663d3e0e36ac6336720286f5a969e04ca3eff273385045ec",
                "online": true,
                "public_address": "0xFbc1d38a2127D81BFe3EA347bec7310a1cfa2373",
                "region": "Quebec-CA",
                "score": 100,
                "status": "Active",
                "updated_at": "1711592195"
            }
        ],
        "jobs": [
            {
                "bidder_id": "0

In [16]:
r = swan_api.get_real_url(task_uuid)
print(r)

['https://5vuov274mo.filezoo.com.cn']


In [18]:
import requests

headers = {
    'Content-Type': 'application/json',
}

json_data = {
    'link': [
        'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYf_0-TvQ0WIDT0wFTAzdqD-c0y1mSfQjfOO0ydStXxA&s',
    ],
}

response = requests.post(f"https://5vuov274mo.filezoo.com.cn/link", headers=headers, json=json_data)
print(response.json())


<Response [200]>
{'result': [{'is_nsfw_image': False, 'is_nsfw_link': False, 'link': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYf_0-TvQ0WIDT0wFTAzdqD-c0y1mSfQjfOO0ydStXxA&s', 'probability': None}]}


In [23]:
log_path = '../nsfw.log'

if os.path.isfile(log_path):
    with open(log_path, 'w') as f:
        f.write('link,IS NSFW IMAGE,IS NSFW LINK,Probability\n')

with open(log_path, 'a') as f:
    for result in response.json()['result']:
        f.write(f"{result['link']},{result['is_nsfw_image']},{result['is_nsfw_link']},{result['probability']}\n")
        